<a href="https://colab.research.google.com/github/axndyg/sagb_inrto_project/blob/main/Shots_Cleaning_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
France = pd.read_csv("/content/events_France.csv")
France.head()

,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id,tagsList,pos_orig_y,pos_orig_x,pos_dest_y,pos_dest_x
0,8,Simple pass,[{'id': 1801}],253784,"[{'y': 51, 'x': 50}, {'y': 46, 'x': 31}]",2500686,Pass,3799,1H,1.935181,85.0,176505119,[1801],51,50,46,31
1,8,High pass,[{'id': 1801}],29474,"[{'y': 46, 'x': 31}, {'y': 74, 'x': 68}]",2500686,Pass,3799,1H,3.599295,83.0,176505121,[1801],46,31,74,68
2,1,Air duel,"[{'id': 703}, {'id': 1801}]",253784,"[{'y': 74, 'x': 68}, {'y': 54, 'x': 72}]",2500686,Duel,3799,1H,6.827043,10.0,176505122,"[703, 1801]",74,68,54,72
3,1,Air duel,"[{'id': 701}, {'id': 1802}]",56441,"[{'y': 26, 'x': 32}, {'y': 46, 'x': 28}]",2500686,Duel,3772,1H,6.985577,10.0,176505017,"[701, 1802]",26,32,46,28
4,1,Ground attacking duel,"[{'id': 702}, {'id': 1801}]",366760,"[{'y': 54, 'x': 72}, {'y': 55, 'x': 73}]",2500686,Duel,3799,1H,9.511272,11.0,176505124,"[702, 1801]",54,72,55,73


In [ ]:
shots_France = France[France["eventName"] == "Shot"]
shots_France = shots_France[['eventId', 'subEventName', 'matchId', 'teamId', 'matchPeriod', 'eventSec']]
shots_France.head()

,eventId,subEventName,matchId,teamId,matchPeriod,eventSec
213,10,Shot,2500686,3799,1H,605.975493
302,10,Shot,2500686,3772,1H,859.236394
498,10,Shot,2500686,3772,1H,1568.104834
577,10,Shot,2500686,3772,1H,1800.852078
629,10,Shot,2500686,3799,1H,2009.537139


In [ ]:
shots_France.loc[shots_France["matchPeriod"] == '2H', 'eventSec'] += 2400

shots_France = shots_France[['eventId', 'subEventName', 'matchId', 'teamId', 'eventSec']]
shots_France['eventMin'] = round(shots_France['eventSec'] / 60)
shots_France.head()


,eventId,subEventName,matchId,teamId,eventSec,eventMin
213,10,Shot,2500686,3799,605.975493,10.0
302,10,Shot,2500686,3772,859.236394,14.0
498,10,Shot,2500686,3772,1568.104834,26.0
577,10,Shot,2500686,3772,1800.852078,30.0
629,10,Shot,2500686,3799,2009.537139,33.0


In [ ]:
by_game = dict((shots_France.groupby("matchId").apply(lambda x: list(dict(x["eventMin"]).values()))))


In [ ]:
shots_cleaned = pd.DataFrame({'matchId': list(by_game.keys()), 'shots': list(by_game.values())})


In [ ]:
shots_per_game = shots_cleaned['shots'].apply(len).tolist()
shots_by_game_late = shots_cleaned['shots'].apply(lambda x: len([i for i in x if i >= 45])).tolist()
shots_by_game_first = shots_cleaned['shots'].apply(lambda x: len([i for i in x if i < 45])).tolist()


In [ ]:
shots_cleaned["total shots"] = shots_per_game
shots_cleaned["shots in second half"] = shots_by_game_late
shots_cleaned["shots in first half"] = shots_by_game_first

shots_cleaned["first half shot density"] =  shots_cleaned["shots in first half"] / shots_cleaned["total shots"]
shots_cleaned["second half shot density"] =  shots_cleaned["shots in second half"] / shots_cleaned["total shots"]

In [ ]:
shots_cleaned

,matchId,shots,total shots,shots in second half,shots in first half,first half shot density,second half shot density
0,2500686,"[10.0, 14.0, 26.0, 30.0, 33.0, 35.0, 42.0, 47....",15,8,7,0.466667,0.533333
1,2500687,"[2.0, 9.0, 14.0, 21.0, 21.0, 24.0, 27.0, 31.0,...",20,9,11,0.550000,0.450000
2,2500688,"[1.0, 3.0, 9.0, 22.0, 23.0, 39.0, 46.0, 49.0, ...",16,10,6,0.375000,0.625000
3,2500689,"[4.0, 11.0, 16.0, 18.0, 19.0, 24.0, 25.0, 26.0...",27,15,12,0.444444,0.555556
4,2500690,"[3.0, 14.0, 18.0, 21.0, 21.0, 32.0, 34.0, 42.0...",14,5,9,0.642857,0.357143
...,...,...,...,...,...,...,...
375,2501061,"[9.0, 12.0, 31.0, 31.0, 32.0, 36.0, 40.0, 44.0...",17,9,8,0.470588,0.529412
376,2501062,"[3.0, 13.0, 15.0, 16.0, 25.0, 26.0, 34.0, 38.0...",16,7,9,0.562500,0.437500
377,2501063,"[4.0, 5.0, 6.0, 6.0, 12.0, 28.0, 29.0, 32.0, 3...",22,10,12,0.545455,0.454545
378,2501064,"[5.0, 9.0, 11.0, 11.0, 20.0, 21.0, 21.0, 22.0,...",26,10,16,0.615385,0.384615


In [ ]:
shots_France.to_csv("shots_France.csv")

In [ ]:
shots_cleaned.to_csv("shots_cleaned.csv")